In [4]:
import re
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import openai
import torch
from dotenv import load_dotenv
from transformers import AutoModelForCausalLM, AutoTokenizer, GPT2LMHeadModel, PreTrainedTokenizerFast
from langchain.chat_models import ChatOpenAI

c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- 사용한 자소서

In [4]:
## 자소서 1
# 입시를 재도전하며 공부했던 기간이 제 인생에 있어 가장 기억에 남는 시간입니다. 과거에 저는 과업을 해결할 때 충분히 준비하지 않고 빠르게 대응하였습니다. 학업을 수행할 때에도 이와 같은 문제가 발생했고, 개념을 제대로 정립하지 않고 문제 풀이에만 집중하는 공부 방식을 택하였습니다. 결국 시험에서 기대치에 미치지 못하는 결과를 받게 되었고 입시에 재도전하게 되었습니다. 부족한 공부 방식을 인식하고, 입시를 재도전한 기간 동안 최대한 계획적이고 규칙적으로 행동하려 노력했습니다. 1년이라는 시간이 계획적인 학습에 충분하다고 믿으며 입시를 위한 공부를 다시 시작하였습니다. 각 과목의 이론을 깊이 이해하면서, 스터디 플래너를 활용하여 계획적으로 공부하였습니다. 하루가 바뀔 때마다 과목별로 세부 공부 목표를 설정하고, 목표를 완료함으로써 공부에 대한 동기부여를 계속 유지할 수 있었습니다. 그렇게 일주일, 한 달의 목표를 성취해 가면서 차근차근 학업의 금자탑을 쌓아 나갔습니다. 312일 동안 꾸준히 노력하고 전략적으로 준비한 결과 원하는 목표를 성취할 수 있었습니다. 이 1년을 통하여 저는 계획적이고 차분하게 행동하는 능력을 키웠습니다. 수험 생활 이후에도 과업을 수행할 때 계획을 먼저 세우고, 충분히 생각한 후에 판단합니다. 모든 과업에 체계적인 접근 방식을 취함으로써 예상을 뛰어넘는 성과를 이뤄냈습니다. 금융업은 고객의 자산을 다루는 업무이기 때문에 정확하고 체계적인 접근이 필요합니다. IBK에 입행한 뒤에도 이때의 경험을 바탕으로 과업에 대한 계획을 수립한 후 천천히 접근할 것입니다. 이후, 상황과 전략에 대한 정확한 분석을 통해 업무를 수행하며 효과적인 결과를 창출하도록 노력할 것입니다.

## 자소서 2 - [계산적으로 업무를 수행하는 인재] - https://linkareer.com/cover-letter/33464?page=1&sort=PASSED_AT&tab=all
# 자소서: 에너지 분야의 적자를 해소하고, 시민들에게 경제적이고 안전한 가스를 공급해드리고 싶어 KOGAS에 지원하게 되었습니다. 물류관리 수업에서 재고와 생산 프로세스 최적화 기법인 'Rock-Drill' 방식을 적용, 실제 산업 사례를 반영한 과제를 성공적으로 수행했습니다. 공급사슬 총괄 계획에서 인적자원, 초과시간, 재고비용, 생산 수를 조절하며 최적의 해를 찾는 연습을 수행했습니다. 요소 간의 균형을 이해하며 분석적 사고 능력을 발전시켰습니다. 나아가 마케팅 캠페인, 규모의 경제, 시간 관리와 같이 불확실한 요소를 고려해 최소 비용 문제를 해결하는 깊이 있는 과제를 완수했습니다. 이렇게 쌓은 제 능력을 가스 수요관리 분야에 적용시킬 수 있습니다. 지역별, 시기별 가스 수요를 조사하여 요소들을 고려하고, 최적의 해를 도출하는 방향으로 활용할 수 있습니다. 이러한 역량을 통해, KOGAS의 적자 해소 및 시민들의 효용 제고에 도움을 주고자 지원했습니다.
# 역량: 수리적/논리적 능력
# 키워드: 분석적 사고, 최적화, 균형, 문제 해결, 효율성
# 비유문장: "저는 여러 요소의 균형을 고려해 최적의 해를 도출하는 분석적인 방정식입니다."

## 자소서 3 - [꼬리에 꼬리를 무는 호기심: 테스트를 하다 알게 된 국제 표준] - https://linkareer.com/cover-letter/33287?page=1&role=%EB%8D%B0%EC%9D%B4%ED%84%B0&sort=PASSED_AT&tab=all
# LG에너지솔루션 / [CTO] 데이터 엔지니어링 / 2023 하반기
# 자소서: 호기심이 많은 성향은 수업을 넘어 실생활에도 적용되었습니다. 그중 컴퓨터의 성능 향상을 위해 메모리 카드를 구매한 뒤, 제품의 성능이 국제 표준과 일치하는지 직접 평가했던 경험은 제품 시험 엔지니어로서의 성장 초석을 다져주었습니다. 인터넷에서 찾은 메모리 카드 테스트 방법에는 전공 수업 때 배운 용어와 개념이 수두룩했습니다. 학교 과제가 아니라, 스스로 전공지식을 제대로 적용하고 배워보고 자동기들을 모아 프로젝트로 추진했습니다. 먼저, 실제 메모리 카드 제조사의 제품 테스트 과정을 벤치마크하기 위해 관련 직무 현직자를 수소문했습니다. 며칠간 연락을 시도한 끝에 운좋게도 현직 제품 시험 엔지니어분과 연락이 닿아 실제 테스트 과정을 배울 수 있었습니다. 이후 상용 프로그램 사용법을 익히고 배운 과정을 실행해 보았습니다. 간이 챔버를 구성해 테스트를 진행하니, 온도 Stress를 주었을 때 특정 주소에서 반복적으로 에러가 발생하는 것을 확인했습니다. 저희 팀은 고온에서 발생할 수 있는 리스크를 조사하여 메모리 카드의 Capacitance가 고온에서 누설 전류가 많아지는 사실을 알았습니다. 이를 바탕으로 고온에서 발생한 에러가 ‘누설 전류에 의한 leakage성 불량일 것이다.’라는 가설을 세웠고, FW 설정에서 Refresh 주기를 절반으로 줄이며 가설을 검증할 수 있었습니다. 이후, 메모리 카드의 국제 표준화 기구의 데이터 시트를 확인하여 제조사가 품질 보증을 위해 준수해야 하는 표준이 존재함을 확인했고, 해당 기준상 특정 온도 이상에서의 불량은 제조사 품질 보증에는 문제가 없음을 확인했습니다. 단순 호기심으로 시작했던 제품 평가가 국제 표준을 확인하고, 데이터 시트를 통해 기준을 충족하는지 분석한 경험으로 이어졌습니다. 배터리 제조사 또한 국제 표준이 존재하고, 이와 같은 경험과 성향은 제품 시험 엔지니어에게 큰 장점이 될 것입니다.
# 역량: 호기심을 바탕으로 한 문제 해결 능력
# 직접 키워드: 호기심, 전공지식, 벤치마킹, 가설 검증, 국제 표준
# 간접 키워드: 자기주도성, 전공지식 응용, 분석력, 탐구심, 실험적 사고

- 개요

In [ ]:
### ⭐ 바꾼 것 ⭐
# 키워드 / 키워드+자기소개서 / 그냥 처음부터 gpt에 (이건 학술적인 게 부족한데 그냥 성능 파악해보려고)
# 비유 문장 여러개 생성하도록
# 설명은 따로 출력하는 게 나을 것 같음.
# 직접 키워드(자소서에 쓰인 단어를 키워드로) / 간접 키워드(내용, 맥락만 동일한 단어를 키워드로) 비교
# temperature=0.9 가 0.7일 때보다 나은 듯 ## temperature = 출력의 다양성을 조절, 숫자 높을수록 더 창의적이고 예측하기 어려운 출력

### ⭐ 프롬프트 ⭐
# 위 자기소개서에서 '호기심을 바탕으로 한 문제 해결 능력'라는 역량에 대해 키워드를 5개 추출해줘. 키워드는 자기소개서에서 직접 추출되지 않아도 돼. 다만, 자기소개서의 내용은 반영하는 단어여야 해. 

### ⭐ 논의사항 ⭐
# 역량을 여러 개 나타내고 싶을 수도 있겠다!
# 역량을 '리더십' 이 아닌, ~를 바탕으로 하는/ 소통과 협업을 중시하는 '협업형 리더십' / 비전 제시형 리더십
# 문제 해결 능력, 호기심을 바탕으로 한 문제 해결 능력
# 비유는 잘 되는데... 메리트가 없는 느낌🚨

In [5]:
#  APIRemovedInV1 오류 해결

## pip uninstall openai
## pip install openai==0.28.1

# **GPT**

<span style = "color : red"> **openai API 없으신 분들은 gpt 파트는 실행시키지 마시고 그냥 확인해주세요! 오류나욧**

In [5]:
load_dotenv() #.env 파일에서 환경 변수 로드
API_KEY = os.environ.get('OPENAI_API_KEY')

## GPT 기본 인퍼런스

In [7]:
def gpt_inference(prompt, model="gpt-4", max_tokens=100):
    response = openai.ChatCompletion.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=60,
        temperature=0.7  # 다양성을 위해 조정 가능
    )
    return response['choices'][0]['message']['content'].strip()

In [8]:
# 테스트용 프롬프트 1
prompt1 = "오늘은 날씨가 좋고, 나는"
generated_sentence1 = gpt_inference(prompt1)
print(generated_sentence1)

## KoGPT와 비교
# 오늘은 날씨가 좋고, 나는 조금 더운날씨가 좋을것같아요.
# 내일은 날이 더워지겠고요.
# 주말에는 대체로 맑은날씨를 회복하겠습니다.
# 오늘도 맑게만 느껴지실텐데요.
# 현재기온은 서울이 15도 선까지 올라있고요.
# 한낮기온도 서울 29도, 대구와 부산 25도까지 오르겠습니다.
# 내일도 전국이 대체로 맑겠지만 동해안지방에는 구름이 많이

산책을 하러 공원에 가기로 결정했다. 가벼운 운동화를 신고, 병에 물을 채워 넣었다. 공원에 도착하니


In [9]:
# 테스트용 프롬프트 2
prompt2 = "저는 여러 아이디어와 의견을 모아 창의적인 해결책을 만들어내는"
generated_sentence2 = gpt_inference(prompt2)
print(generated_sentence2)

## KoGPT와 비교
# 저는 여러 아이디어와 의견을 모아 창의적인 해결책을 만들어내는 것이 중요하다는 생각을 하게 되었습니다.
# 여러분들의 아이디어를 토대로 좋은 방안을 만들고 싶습니다.
# 최근에 저를 포함한 많은 분들이 새로운 아이디어를 내신 적이 있습니다.
# 새로운 아이디어를 갖고 계신 분들에게 저의 아이디어가 어떤 것인지 물어보시기 바랍니다.
# 여러분의 아이디어는 여러분들 각자의 개개인의 재능과 능력에 따라 달라질 수 있습니다.
# 여러분이 지금 당장 여러분이 가진 창의력과 잠재력을 최대한 발휘할 수

데 뛰어납니다. 다양한 관점에서 문제를 바라보고, 합리적이면서도 창의적인 방법으로 문제를 해결하는 능력을 가지고 있습니다.


## GPT 기본 프롬프트

### 1. 키워드만 입력

In [56]:
# 비유 문장과 설명을 함께 여러 개 생성하는 함수
def gpt_keywords(keywords):
    
    # OpenAI GPT-4 Chat API를 사용하여 프롬프트를 전달
    prompt = f"""
    주어진 키워드를 바탕으로, 자기소개서를 쓴 사람을 나타낼 수 있는 비유 문장 5개를 각각 '저는 ~입니다.' 형식으로 작성하고, 왜 그 비유를 사용했는지 구체적으로 설명해주세요.
    비유된 대상(예: 나침반)이 선택된 이유와 그 대상이 어떻게 주어진 키워드들과 연결되는지를 완성된 문장으로 설명해주세요.
    각 비유 문장은 반드시 '저는 ~입니다.'로 시작해야 하며, 간결하게 작성되어야 합니다. 총 5개의 비유 문장과 그에 대한 설명을 각각 작성해주세요. 
    문장은 반드시 하나의 문장으로 완성되어야 합니다.

    예시:
    - 비유 문장: 저는 어떠한 문제도 분석하고 해결할 수 있는 확고한 방향성을 가진 나침반입니다.
      설명: 나침반은 방향을 제시하고, 문제 해결 능력은 올바른 방향을 찾아내는 것과 같기 때문에 나침반으로 비유했습니다. 나침반은 모든 상황에서 일관되게 목표를 향해 움직일 수 있는 도구로, 이 능력은 복잡한 문제 속에서도 명확한 방향을 찾는 능력과 일치합니다.

    - 비유 문장: 저는 여러 아이디어와 의견을 모아 창의적인 해결책을 만들어내는 팔레트입니다.
      설명: 팔레트는 다양한 색을 혼합해 새로운 작품을 창조하듯, 여러 가지 아이디어를 결합해 새로운 해결책을 만들어내는 능력을 표현합니다. 이 비유는 팔레트의 창의적인 역할이 다양한 의견을 효과적으로 융합하는 능력과 연관되기 때문에 사용되었습니다.
    
    5개의 비유 문장과 그에 대한 설명을 각각 작성해주세요. 각 문장들은 반드시 완전한 문장으로 끝나야 합니다.
    """
    
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "당신은 입력된 키워드를 바탕으로 비유 문장을 생성하는 전문가입니다."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=700,  # 다섯 개의 비유 문장과 설명을 위해 토큰 수 확장
        temperature=0.9,  # 출력의 다양성을 조절 - 숫자 높을수록 더 창의적이고 예측하기 어려운 출력
        top_p=1.0,
        stop=["\n"]  # 두 줄 공백이 나오면 멈추도록 함 (각 문장이 자연스럽게 끝나도록)
    )

    # 결과 출력
    metaphor_sentences = response['choices'][0]['message']['content'].strip()
    return metaphor_sentences

- 자소서 1

In [ ]:
# 자소서 1
# 자소서: 에너지 분야의 적자를 해소하고, 시민들에게 경제적이고 안전한 가스를 공급해드리고 싶어 KOGAS에 지원하게 되었습니다. 물류관리 수업에서 재고와 생산 프로세스 최적화 기법인 'Rock-Drill' 방식을 적용, 실제 산업 사례를 반영한 과제를 성공적으로 수행했습니다. 공급사슬 총괄 계획에서 인적자원, 초과시간, 재고비용, 생산 수를 조절하며 최적의 해를 찾는 연습을 수행했습니다. 요소 간의 균형을 이해하며 분석적 사고 능력을 발전시켰습니다. 나아가 마케팅 캠페인, 규모의 경제, 시간 관리와 같이 불확실한 요소를 고려해 최소 비용 문제를 해결하는 깊이 있는 과제를 완수했습니다. 이렇게 쌓은 제 능력을 가스 수요관리 분야에 적용시킬 수 있습니다. 지역별, 시기별 가스 수요를 조사하여 요소들을 고려하고, 최적의 해를 도출하는 방향으로 활용할 수 있습니다. 이러한 역량을 통해, KOGAS의 적자 해소 및 시민들의 효용 제고에 도움을 주고자 지원했습니다.
# 역량: 수리적/논리적 능력

In [ ]:
# temperature=0.9, stop=["\n\n"]

keywords = ["분석적 사고", "최적화", "균형", "문제 해결", "효율성"]
sentence = gpt_keywords(keywords)
print(sentence)

- 비유 문장: 저는 어떤 상황에도 대처할 수 있는 준비된 백신입니다.


In [58]:
def gpt_keywords(keywords):
    
    # OpenAI GPT-4 Chat API를 사용하여 프롬프트를 전달
    prompt = f"""
    주어진 키워드: {keywords}
    위의 키워드를 바탕으로, 자기소개서를 쓴 사람을 나타낼 수 있는 비유 문장 5개를 각각 '저는 ~입니다.' 형식으로 작성하고, 왜 그 비유를 사용했는지 구체적으로 설명해주세요.
    비유된 대상(예: 나침반)이 선택된 이유와 그 대상이 어떻게 주어진 키워드들과 자기소개서 내용과 연결되는지를 완성된 문장으로 설명해주세요.
    각 비유 문장은 반드시 '저는 ~입니다.'로 시작해야 하며, 간결하게 작성되어야 합니다. 총 5개의 문장을 각각 작성해주세요.
    문장은 반드시 하나의 문장으로 완성되어야 합니다.

    예시:
    - 비유 문장: 저는 어떠한 문제도 분석하고 해결할 수 있는 확고한 방향성을 가진 나침반입니다.
      설명: 나침반은 방향을 제시하고, 문제 해결 능력은 올바른 방향을 찾아내는 것과 같기 때문에 나침반으로 비유했습니다. 나침반은 모든 상황에서 일관되게 목표를 향해 움직일 수 있는 도구로, 이 능력은 복잡한 문제 속에서도 명확한 방향을 찾는 능력과 일치합니다.

    - 비유 문장: 저는 여러 아이디어와 의견을 모아 창의적인 해결책을 만들어내는 팔레트입니다.
      설명: 팔레트는 다양한 색을 혼합해 새로운 작품을 창조하듯, 여러 가지 아이디어를 결합해 새로운 해결책을 만들어내는 능력을 표현합니다. 이 비유는 팔레트의 창의적인 역할이 다양한 의견을 효과적으로 융합하는 능력과 연관되기 때문에 사용되었습니다.
    
    5개의 비유 문장과 그에 대한 설명을 각각 작성해주세요. 각 문장들은 반드시 완전한 문장으로 끝나야 해.
    """
  
    
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "당신은 입력된 키워드와 자기소개서를 바탕으로 비유 문장을 생성하는 전문가입니다."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=800,  # 다섯 개의 비유 문장과 설명을 위해 토큰 수 확장
        temperature=0.9,
        top_p=1.0,
    )

    # 결과 출력
    metaphor_sentences = response['choices'][0]['message']['content'].strip()
    return metaphor_sentences

- 자소서 1

In [ ]:
# 자소서 1
# 자소서: 에너지 분야의 적자를 해소하고, 시민들에게 경제적이고 안전한 가스를 공급해드리고 싶어 KOGAS에 지원하게 되었습니다. 물류관리 수업에서 재고와 생산 프로세스 최적화 기법인 'Rock-Drill' 방식을 적용, 실제 산업 사례를 반영한 과제를 성공적으로 수행했습니다. 공급사슬 총괄 계획에서 인적자원, 초과시간, 재고비용, 생산 수를 조절하며 최적의 해를 찾는 연습을 수행했습니다. 요소 간의 균형을 이해하며 분석적 사고 능력을 발전시켰습니다. 나아가 마케팅 캠페인, 규모의 경제, 시간 관리와 같이 불확실한 요소를 고려해 최소 비용 문제를 해결하는 깊이 있는 과제를 완수했습니다. 이렇게 쌓은 제 능력을 가스 수요관리 분야에 적용시킬 수 있습니다. 지역별, 시기별 가스 수요를 조사하여 요소들을 고려하고, 최적의 해를 도출하는 방향으로 활용할 수 있습니다. 이러한 역량을 통해, KOGAS의 적자 해소 및 시민들의 효용 제고에 도움을 주고자 지원했습니다.
# 역량: 수리적/논리적 능력

In [59]:
# temperature=0.9

keywords = ["분석적 사고", "최적화", "균형", "문제 해결", "효율성"]
sentence = gpt_keywords(keywords)
print(sentence)

- 비유 문장: 저는 정보를 분석하고 해결책을 찾아내는 프로그래머입니다.
      설명: 프로그래머는 문제를 분석하고 코드를 짜서 해결책을 찾아내는 일을 합니다. 이는 제 '분석적 사고'와 '문제 해결' 능력을 잘 나타냅니다.

- 비유 문장: 저는 무게를 균형있게 유지하는 저울입니다.
      설명: 저울은 무게를 균형있게 유지하는 데에 목적이 있습니다. 이는 제가 팀 프로젝트에서 여러 의견을 공평하게 듣고 균형있는 판단을 내릴 수 있는 능력을 의미합니다.

- 비유 문장: 저는 최적의 경로를 찾아내는 GPS입니다.
      설명: GPS는 사용자에게 최적의 경로를 찾아 제공하는 기능을 합니다. 이는 제가 주어진 자원과 시간 안에서 최적의 결과를 도출해내는 '최적화' 능력을 상징합니다.

- 비유 문장: 저는 작업의 효율성을 높이는 컨베이어 벨트입니다.
      설명: 컨베이어 벨트는 물건을 빠르고 효율적으로 이동시키는 역할을 합니다. 이는 제가 작업을 체계적이고 효율적으로 처리하여 목표를 달성하는 능력을 상징합니다.

- 비유 문장: 저는 복잡한 문제를 해결하는 Rubik's Cube 솔버입니다.
      설명: Rubik's Cube 솔버는 복잡한 큐브 문제를 해결하는 방법을 찾아내는 능력을 가지고 있습니다. 이는 제가 복잡한 문제를 분석하고 해결할 수 있는 '문제 해결' 능력을 의미합니다.


In [ ]:
# temperature=0.7, stop=["\n\n"]

keywords = ["분석적 사고", "최적화", "균형", "문제 해결", "효율성"]
sentence = gpt_keywords(keywords)
print(sentence)

- 비유 문장: 저는 언제나 변화에 적응하고 성장할 준비가 된 숲의 나무입니다.
  설명: 숲의 나무는 계절과 환경의 변화에 적응하며 성장하는 특성을 가지고 있습니다. 이는 변화에 빠르게 적응하고, 새로운 환경에서도 성장할 수 있는 능력을 표현하기 위해 선택된 비유입니다.

- 비유 문장: 저는 항상 끊임없이 노력하고 도전하는 물결입니다.
  설명: 물결은 항상 움직이며 끊임없이 해안을 밀고 온다는 점에서, 노력과 도전을 계속하는 특성을 가진다고 할 수 있습니다. 이러한 비유는 끊임없이 노력하고 도전하는 자세를 표현하기 위해 사용되었습니다.

- 비유 문장: 저는 무엇이든지 받아들이며, 그것을 자신의 것으로 만들 수 있는 스펀지입니다.
  설명: 스펀지는 물이든, 공기든 무엇이든 흡수하며 그것을 자신의 일부로 만드는 특성을 가지고 있습니다. 이 비유는 새로운 정보나 지식을 받아들이고 이해하는 능력을 표현하기 위해 사용되었습니다.

- 비유 문장: 저는 모두가 행복을 찾을 수 있도록 돕는 가이드북입니다.
  설명: 가이드북은 사람들이 목적지를 찾고, 그곳에서 행복을 느낄 수 있도록 돕


- 자소서 2

In [ ]:
# 자소서 2
# 자소서: 호기심이 많은 성향은 수업을 넘어 실생활에도 적용되었습니다. 그중 컴퓨터의 성능 향상을 위해 메모리 카드를 구매한 뒤, 제품의 성능이 국제 표준과 일치하는지 직접 평가했던 경험은 제품 시험 엔지니어로서의 성장 초석을 다져주었습니다. 인터넷에서 찾은 메모리 카드 테스트 방법에는 전공 수업 때 배운 용어와 개념이 수두룩했습니다. 학교 과제가 아니라, 스스로 전공지식을 제대로 적용하고 배워보고 자동기들을 모아 프로젝트로 추진했습니다. 먼저, 실제 메모리 카드 제조사의 제품 테스트 과정을 벤치마크하기 위해 관련 직무 현직자를 수소문했습니다. 며칠간 연락을 시도한 끝에 운좋게도 현직 제품 시험 엔지니어분과 연락이 닿아 실제 테스트 과정을 배울 수 있었습니다. 이후 상용 프로그램 사용법을 익히고 배운 과정을 실행해 보았습니다. 간이 챔버를 구성해 테스트를 진행하니, 온도 Stress를 주었을 때 특정 주소에서 반복적으로 에러가 발생하는 것을 확인했습니다. 저희 팀은 고온에서 발생할 수 있는 리스크를 조사하여 메모리 카드의 Capacitance가 고온에서 누설 전류가 많아지는 사실을 알았습니다. 이를 바탕으로 고온에서 발생한 에러가 ‘누설 전류에 의한 leakage성 불량일 것이다.’라는 가설을 세웠고, FW 설정에서 Refresh 주기를 절반으로 줄이며 가설을 검증할 수 있었습니다. 이후, 메모리 카드의 국제 표준화 기구의 데이터 시트를 확인하여 제조사가 품질 보증을 위해 준수해야 하는 표준이 존재함을 확인했고, 해당 기준상 특정 온도 이상에서의 불량은 제조사 품질 보증에는 문제가 없음을 확인했습니다. 단순 호기심으로 시작했던 제품 평가가 국제 표준을 확인하고, 데이터 시트를 통해 기준을 충족하는지 분석한 경험으로 이어졌습니다. 배터리 제조사 또한 국제 표준이 존재하고, 이와 같은 경험과 성향은 제품 시험 엔지니어에게 큰 장점이 될 것입니다.
# 역량: 호기심을 바탕으로 한 문제 해결 능력

In [60]:
# temperature=0.9

# 직접 키워드:
keywords = ["호기심", "전공지식", "벤치마킹", "가설 검증", "국제 표준"]
sentence = gpt_keywords(keywords)
print("직접 키워드 사용:\n", sentence)
print("")
print("")
# 간접 키워드:
keywords = ["자기주도성", "전공지식 응용", "분석력", "탐구심", "실험적 사고"]
sentence = gpt_keywords(keywords)
print("간접 키워드 사용:\n", sentence)

직접 키워드 사용:
 1. 비유 문장: 저는 호기심을 통해 길을 찾는 등산객입니다.
   설명: 등산객은 알려지지 않은 등산로로의 모험을 두려워하지 않고, 오히려 그것을 즐기며, 호기심에 따라 길을 찾습니다. 저는 이러한 등산객처럼 언제나 호기심을 가지고 새로운 정보와 지식을 탐구하고 찾아나갑니다.

2. 비유 문장: 저는 깊은 바다 속에 숨겨진 진주처럼 깊은 전공지식을 가지고 있습니다.
   설명: 진주는 깊은 해저에 깔려 있는 것처럼, 제 전공지식 또한 깊은 이해와 심도를 바탕으로 깔려 있는 것입니다. 이러한 비유는 제가 가진 전공에 대한 깊은 이해와 학문적 통찰력을 보여줍니다.

3. 비유 문장: 저는 다른 기업의 성공 사례를 자신의 것으로 만들 수 있는 벤치마킹 전문가처럼 실용적입니다.
   설명: 벤치마킹은 성공적인 사례를 분석하고 이를 자신의 경우에 적용하는 것입니다. 저는 이와 같이 다른 사람이나 조직의 좋은 점을 배울 줄 아는 사람이며, 그것들을 자신의 사례에 적용하는 능력을 가졌습니다.

4. 비유 문장: 저는 체계적인 가설 검증을 통해 문제를 해결하는 과학자입니다.
   설명: 과학자는 체계적인 방법을 통해 문제를 분석하고 가설을 검증합니다. 저는 이와 같이 체계적인 사고를 바탕으로 문제를 해결하는 능력을 가졌습니다.

5. 비유 문장: 저는 국제 표준을 준수하는 세계 시민입니다.
   설명: 세계 시민은 국제적인 표준과 규칙을 인식하고 이를 준수합니다. 저는 이와 같이 국제적인 관점을 가지고 있으며, 이를 바탕으로 표준 및 규칙을 준수하는 능력을 가졌습니다.


간접 키워드 사용:
 - 비유 문장: 저는 스스로 목표를 찾고 그것을 적극적으로 추구하는 자기주도성을 가진 나침반입니다.
  설명: 나침반은 방향을 향해 자유롭게 움직이는 도구이며, 이는 자기주도성을 가진 사람이 자신의 목표를 향해 적극적으로 움직이는 것과 비슷합니다. 따라서, 자기주도성을 나타내는데 나침반을 비유로 사용했습니다.

- 비유 문장: 저는 전공 지식을 실질적인 문제에 

In [ ]:
# temperature=0.7

# 직접 키워드:
keywords = ["호기심", "전공지식", "벤치마킹", "가설 검증", "국제 표준"]
sentence = gpt_keywords(keywords)
print("직접 키워드 사용:\n", sentence)
print("")
print("")
# 간접 키워드:
keywords = ["자기주도성", "전공지식 응용", "분석력", "탐구심", "실험적 사고"]
sentence = gpt_keywords(keywords)
print("간접 키워드 사용:\n", sentence)

직접 키워드 사용:
 - 비유 문장: 저는 끊임없이 배우고 성장하는 식물입니다.
  설명: 식물은 지속적으로 물과 햇빛을 통해 성장하고, 때론 어려운 환경에도 적응해 나가는 특성을 가지고 있습니다. 이와 같이, 저는 새로운 것을 배우고, 경험을 통해 계속해서 자신을 발전시키는 사람을 의미합니다.

- 비유 문장: 저는 많은 정보와 사람들을 잇는 하나의 다리입니다.
  설명: 다리는 한 곳에서 다른 곳으로 이동하게 해주는 역할을 수행합니다. 저는 다양한 정보와 사람들 사이에서 연결 역할을 하는 사람으로, 다리와 같은 역할을 수행합니다.

- 비유 문장: 저는 팀의 목표를 이루기 위해 모두를 돕는 앵커입니다.
  설명: 앵커는 배를 움직이지 않게 고정시키는 역할을 합니다. 저는 팀의 목표를 달성하기 위해 필요한 어떤 역할이든 수행하며, 팀원들이 목표를 향해 나아갈 수 있도록 돕는 사람입니다.

- 비유 문장: 저는 어떤 상황에서도 긍정적인 에너지를 내뿜는 태양입니다.
  설명: 태양은 언제나 밝고 긍정적인 에너지를 내뿜습니다. 이와 같이 저는 어떤 상황에서도 긍정적인 마음을 유지하고, 그 에너지를 주변 사람들에게 전달하는 사람입니다.

- 비유 문장: 저는 목표를 향해 끊임없이 나아가는 나침반입니다.
  설명: 나침반은 항상 한 방향을 향해 나아가는 도구입니다. 저는 목표를 설정하고, 그 목표를 향해 끊임없이 노력하며 성장하는 사람을 의미합니다.


간접 키워드 사용:
 - 비유 문장: 저는 매 순간 최선의 선택을 내리는 체스마스터입니다.
  설명: 체스마스터는 게임의 모든 단계에서 최적의 전략을 설정하고 판단력을 발휘해 승리를 이끌어내는 역할을 합니다. 이는 복잡한 상황에서도 올바른 결정을 내릴 수 있는 능력과 맞닿아 있으므로 체스마스터로 비유했습니다.

- 비유 문장: 저는 어떤 환경에서도 생존하고, 성장하는 능력을 가진 야생화입니다.
  설명: 야생화는 가혹한 환경에서도 살아남고 성장하는 능력을 가지고 있습니다. 이는 어려운 상황에서도 흔들리지 않고 성장하려

### 2. 키워드 & 자기소개서 입력

In [40]:
# 다 temperature=0.9

In [48]:
def gpt_keywords_essay(keywords, essay):
    
    # OpenAI GPT-4 Chat API를 사용하여 프롬프트를 전달
    prompt = f"""
    주어진 키워드: {keywords}
    자기소개서 내용: {essay}
    위의 키워드와 자기소개서를 바탕으로, 자기소개서를 쓴 사람을 나타낼 수 있는 비유 문장 5개를 각각 '저는 ~입니다.' 형식으로 작성하고, 왜 그 비유를 사용했는지 구체적으로 설명해주세요.
    비유된 대상(예: 나침반)이 선택된 이유와 그 대상이 어떻게 주어진 키워드들과 자기소개서 내용과 연결되는지를 완성된 문장으로 설명해주세요.
    각 비유 문장은 반드시 '저는 ~입니다.'로 시작해야 하며, 간결하게 작성되어야 합니다. 총 5개의 문장을 각각 작성해주세요.
    문장은 반드시 하나의 문장으로 완성되어야 합니다.

    예시:
    - 비유 문장: 저는 어떠한 문제도 분석하고 해결할 수 있는 확고한 방향성을 가진 나침반입니다.
      설명: 나침반은 방향을 제시하고, 문제 해결 능력은 올바른 방향을 찾아내는 것과 같기 때문에 나침반으로 비유했습니다. 나침반은 모든 상황에서 일관되게 목표를 향해 움직일 수 있는 도구로, 이 능력은 복잡한 문제 속에서도 명확한 방향을 찾는 능력과 일치합니다.

    - 비유 문장: 저는 여러 아이디어와 의견을 모아 창의적인 해결책을 만들어내는 팔레트입니다.
      설명: 팔레트는 다양한 색을 혼합해 새로운 작품을 창조하듯, 여러 가지 아이디어를 결합해 새로운 해결책을 만들어내는 능력을 표현합니다. 이 비유는 팔레트의 창의적인 역할이 다양한 의견을 효과적으로 융합하는 능력과 연관되기 때문에 사용되었습니다.
    
    5개의 비유 문장과 그에 대한 설명을 각각 작성해주세요. 각 문장들은 반드시 완전한 문장으로 끝나야 해.
    """
  
    
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "당신은 입력된 키워드와 자기소개서를 바탕으로 비유 문장을 생성하는 전문가입니다."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=800,  # 다섯 개의 비유 문장과 설명을 위해 토큰 수 확장
        temperature=0.9,
        top_p=1.0,
    )

    # 결과 출력
    metaphor_sentences = response['choices'][0]['message']['content'].strip()
    return metaphor_sentences

- 자소서 1

In [ ]:
# 자소서 1
# 자소서: 에너지 분야의 적자를 해소하고, 시민들에게 경제적이고 안전한 가스를 공급해드리고 싶어 KOGAS에 지원하게 되었습니다. 물류관리 수업에서 재고와 생산 프로세스 최적화 기법인 'Rock-Drill' 방식을 적용, 실제 산업 사례를 반영한 과제를 성공적으로 수행했습니다. 공급사슬 총괄 계획에서 인적자원, 초과시간, 재고비용, 생산 수를 조절하며 최적의 해를 찾는 연습을 수행했습니다. 요소 간의 균형을 이해하며 분석적 사고 능력을 발전시켰습니다. 나아가 마케팅 캠페인, 규모의 경제, 시간 관리와 같이 불확실한 요소를 고려해 최소 비용 문제를 해결하는 깊이 있는 과제를 완수했습니다. 이렇게 쌓은 제 능력을 가스 수요관리 분야에 적용시킬 수 있습니다. 지역별, 시기별 가스 수요를 조사하여 요소들을 고려하고, 최적의 해를 도출하는 방향으로 활용할 수 있습니다. 이러한 역량을 통해, KOGAS의 적자 해소 및 시민들의 효용 제고에 도움을 주고자 지원했습니다.
# 역량: 수리적/논리적 능력

In [49]:
essay = "에너지 분야의 적자를 해소하고, 시민들에게 경제적이고 안전한 가스를 공급해드리고 싶어 KOGAS에 지원하게 되었습니다. 물류관리 수업에서 재고와 생산 프로세스 최적화 기법인 'Rock-Drill' 방식을 적용, 실제 산업 사례를 반영한 과제를 성공적으로 수행했습니다. 공급사슬 총괄 계획에서 인적자원, 초과시간, 재고비용, 생산 수를 조절하며 최적의 해를 찾는 연습을 수행했습니다. 요소 간의 균형을 이해하며 분석적 사고 능력을 발전시켰습니다. 나아가 마케팅 캠페인, 규모의 경제, 시간 관리와 같이 불확실한 요소를 고려해 최소 비용 문제를 해결하는 깊이 있는 과제를 완수했습니다. 이렇게 쌓은 제 능력을 가스 수요관리 분야에 적용시킬 수 있습니다. 지역별, 시기별 가스 수요를 조사하여 요소들을 고려하고, 최적의 해를 도출하는 방향으로 활용할 수 있습니다. 이러한 역량을 통해, KOGAS의 적자 해소 및 시민들의 효용 제고에 도움을 주고자 지원했습니다."
keywords = ["분석적 사고", "최적화", "균형", "문제 해결", "효율성"]

sentence = gpt_keywords_essay(keywords, essay)
print(sentence)

1. 비유 문장: 저는 조화를 이루며 끊임없이 균형을 최적화하는 바퀴입니다.
   설명: 바퀴는 여러 부품이 조화롭게 움직여 원활한 동작을 이루는 대표적인 기구로, 이 비유는 자기소개서에서 언급된 균형과 최적화의 중요성을 대변합니다. 조화를 이루며 돌아가는 바퀴와 같이 평형을 이루는 능력을 표현할 수 있습니다.

2. 비유 문장: 저는 효율적인 경로를 따르는 물류 관리자입니다.
   설명: 물류 관리자는 최적의 경로를 찾아 비용을 절약하고 효율성을 높이는 업무를 수행하는데, 이는 자기소개서에서 언급된 '분석적 사고'와 '최적화', '효율성'의 키워드에 연결됩니다.

3. 비유 문장: 저는 여러 변수들간의 균형을 잡는 저울입니다.
   설명: 저울은 여러 무게를 적절히 분포시켜 균형을 이루는 기능으로, 이 비유는 자기소개서의 '균형'이라는 키워드와 연결됩니다. 여러 변수들 사이에서 균형을 잡아 나가는 능력을 표현합니다.

4. 비유 문장: 저는 복잡한 문제를 풀어내는 퍼즐의 마스터입니다.
   설명: 퍼즐의 마스터는 복잡한 퍼즐을 분석하고 그것을 해결하는 능력을 가지고 있습니다. 이 비유는 '분석적 사고'와 '문제 해결'이라는 키워드와 연결되며, 복잡한 문제를 분석하여 해결하는 능력을 나타냅니다.

5. 비유 문장: 저는 여러 요소를 조합해 최적의 결과를 도출하는 요리사입니다.
   설명: 요리사는 다양한 재료를 조합하고, 그 중에서 가장 맛있는 요리를 만드는 것에 집중합니다. 이 비유는 '최적화'와 '효율성'이라는 키워드와 연결되며, 여러 요소를 고려하여 최적의 결과를 도출하는 능력을 나타냅니다.


- 자소서 2

In [ ]:
# 자소서 2
# 자소서: 호기심이 많은 성향은 수업을 넘어 실생활에도 적용되었습니다. 그중 컴퓨터의 성능 향상을 위해 메모리 카드를 구매한 뒤, 제품의 성능이 국제 표준과 일치하는지 직접 평가했던 경험은 제품 시험 엔지니어로서의 성장 초석을 다져주었습니다. 인터넷에서 찾은 메모리 카드 테스트 방법에는 전공 수업 때 배운 용어와 개념이 수두룩했습니다. 학교 과제가 아니라, 스스로 전공지식을 제대로 적용하고 배워보고 자동기들을 모아 프로젝트로 추진했습니다. 먼저, 실제 메모리 카드 제조사의 제품 테스트 과정을 벤치마크하기 위해 관련 직무 현직자를 수소문했습니다. 며칠간 연락을 시도한 끝에 운좋게도 현직 제품 시험 엔지니어분과 연락이 닿아 실제 테스트 과정을 배울 수 있었습니다. 이후 상용 프로그램 사용법을 익히고 배운 과정을 실행해 보았습니다. 간이 챔버를 구성해 테스트를 진행하니, 온도 Stress를 주었을 때 특정 주소에서 반복적으로 에러가 발생하는 것을 확인했습니다. 저희 팀은 고온에서 발생할 수 있는 리스크를 조사하여 메모리 카드의 Capacitance가 고온에서 누설 전류가 많아지는 사실을 알았습니다. 이를 바탕으로 고온에서 발생한 에러가 ‘누설 전류에 의한 leakage성 불량일 것이다.’라는 가설을 세웠고, FW 설정에서 Refresh 주기를 절반으로 줄이며 가설을 검증할 수 있었습니다. 이후, 메모리 카드의 국제 표준화 기구의 데이터 시트를 확인하여 제조사가 품질 보증을 위해 준수해야 하는 표준이 존재함을 확인했고, 해당 기준상 특정 온도 이상에서의 불량은 제조사 품질 보증에는 문제가 없음을 확인했습니다. 단순 호기심으로 시작했던 제품 평가가 국제 표준을 확인하고, 데이터 시트를 통해 기준을 충족하는지 분석한 경험으로 이어졌습니다. 배터리 제조사 또한 국제 표준이 존재하고, 이와 같은 경험과 성향은 제품 시험 엔지니어에게 큰 장점이 될 것입니다.
# 역량: 호기심을 바탕으로 한 문제 해결 능력

In [50]:
essay = "호기심이 많은 성향은 수업을 넘어 실생활에도 적용되었습니다. 그중 컴퓨터의 성능 향상을 위해 메모리 카드를 구매한 뒤, 제품의 성능이 국제 표준과 일치하는지 직접 평가했던 경험은 제품 시험 엔지니어로서의 성장 초석을 다져주었습니다. 인터넷에서 찾은 메모리 카드 테스트 방법에는 전공 수업 때 배운 용어와 개념이 수두룩했습니다. 학교 과제가 아니라, 스스로 전공지식을 제대로 적용하고 배워보고 자동기들을 모아 프로젝트로 추진했습니다. 먼저, 실제 메모리 카드 제조사의 제품 테스트 과정을 벤치마크하기 위해 관련 직무 현직자를 수소문했습니다. 며칠간 연락을 시도한 끝에 운좋게도 현직 제품 시험 엔지니어분과 연락이 닿아 실제 테스트 과정을 배울 수 있었습니다. 이후 상용 프로그램 사용법을 익히고 배운 과정을 실행해 보았습니다. 간이 챔버를 구성해 테스트를 진행하니, 온도 Stress를 주었을 때 특정 주소에서 반복적으로 에러가 발생하는 것을 확인했습니다. 저희 팀은 고온에서 발생할 수 있는 리스크를 조사하여 메모리 카드의 Capacitance가 고온에서 누설 전류가 많아지는 사실을 알았습니다. 이를 바탕으로 고온에서 발생한 에러가 ‘누설 전류에 의한 leakage성 불량일 것이다.’라는 가설을 세웠고, FW 설정에서 Refresh 주기를 절반으로 줄이며 가설을 검증할 수 있었습니다. 이후, 메모리 카드의 국제 표준화 기구의 데이터 시트를 확인하여 제조사가 품질 보증을 위해 준수해야 하는 표준이 존재함을 확인했고, 해당 기준상 특정 온도 이상에서의 불량은 제조사 품질 보증에는 문제가 없음을 확인했습니다. 단순 호기심으로 시작했던 제품 평가가 국제 표준을 확인하고, 데이터 시트를 통해 기준을 충족하는지 분석한 경험으로 이어졌습니다. 배터리 제조사 또한 국제 표준이 존재하고, 이와 같은 경험과 성향은 제품 시험 엔지니어에게 큰 장점이 될 것입니다."

# 직접 키워드:
keywords = ["호기심", "전공지식", "벤치마킹", "가설 검증", "국제 표준"]
sentence = gpt_keywords_essay(keywords, essay)
print("직접 키워드 사용:\n", sentence)
print("")
print("")
# 간접 키워드:
keywords = ["자기주도성", "전공지식 응용", "분석력", "탐구심", "실험적 사고"]
sentence = gpt_keywords_essay(keywords, essay)
print("간접 키워드 사용:\n", sentence)

직접 키워드 사용:
 1. 비유 문장: 저는 호기심을 바탕으로 새로운 세계를 탐험하는 모험가입니다.
   설명: 모험가는 알려지지 않은 세계를 탐험하고 발견하는 일에 즐거움을 느끼는 사람으로, 이것을 자기소개서에서의 '호기심' 키워드와 연관지었습니다. 저의 호기심은 메모리 카드의 성능 향상에 대한 탐색을 이끌었고, 이를 통해 전공지식을 활용하며 실생활에 적용하는 과정을 경험하였습니다.

2. 비유 문장: 저는 전공지식을 바탕으로 다양한 문제를 해결하는 툴박스입니다. 
   설명: 툴박스는 다양한 도구를 갖추고 있어 여러 문제를 해결하는데 필요한 도구로, 이는 '전공지식' 키워드와 연결됩니다. 저의 전공지식은 제품의 성능을 평가하고, 에러 현상을 분석하며, 결국 문제를 해결하는 데 필요한 도구과 같습니다. 

3. 비유 문장: 저는 다른 사람들의 경험과 지식을 자신의 것으로 만들어 성장하는 스펀지입니다.
   설명: 스펀지는 주변의 물질을 빠르게 흡수할 수 있는 능력이 있습니다. 이것은 '벤치마킹' 키워드를 통해 나타나는 능력을 표현합니다. 저는 직장인의 테스트 과정을 벤치마킹 하여 자신의 것으로 만드는 과정에서 스펀지처럼 다양한 지식과 경험을 빠르게 흡수하고 성장하였습니다.

4. 비유 문장: 저는 새로운 가설을 세우고 검증하는 과학자입니다.
   설명: 과학자는 새로운 가설을 세우고 실험을 통해 그 가설을 검증합니다. 이는 '가설 검증' 키워드와 일치하며, 제가 메모리 카드의 불량 원인을 가설로 세우고 이를 테스트로 검증한 경험을 적절하게 표현합니다.

5. 비유 문장: 저는 국제 표준에 부합하는 제품을 평가하는 검사관입니다.
   설명: 검사관은 특정 기준에 부합하는지를 검사하는 역할을 합니다. 이는 '국제 표준'


간접 키워드 사용:
 1. 비유 문장: 저는 항상 새로운 시도를 두려워하지 않는 실험대입니다.
   설명: 실험대는 새로운 시도와 실험에서 중요한 역할을 하는 도구로, 저의 실험적 사고와 자기주도성을 비유하기에 적합합니다. 실험대는 불확실한 

### 3. 역량 & 자기소개서 입력

In [6]:
def gpt_competency_essay(competency, essay):
    
    # GPT-4에 전달할 프롬프트
    prompt = f"""
    [역량]: {competency}
    [자기소개서 내용]: {essay}
    
    위의 역량과 자기소개서 내용을 바탕으로, 자기소개서를 쓴 사람을 나타낼 수 있는 비유 문장을 5개 작성하고, 각 비유 문장에 대해 왜 그 비유를 선택했는지 설명해주세요.
    각 문장은 '저는 ~입니다.' 형식으로 작성되어야 하며, 비유된 대상이 역량과 어떻게 연결되는지 구체적으로 설명해주세요.
    각 문장은 하나의 문장으로 완성되어야 하며, 설명은 문장 뒤에 추가되어야 합니다.

    예시:
    - 비유 문장: 저는 어떠한 문제도 분석하고 해결할 수 있는 확고한 방향성을 가진 나침반입니다.
      설명: 나침반은 방향을 제시하고, 문제 해결 능력은 올바른 방향을 찾아내는 것과 같기 때문에 나침반으로 비유했습니다.
    
    - 비유 문장: 저는 여러 아이디어와 의견을 모아 창의적인 해결책을 만들어내는 팔레트입니다.
      설명: 팔레트는 다양한 색을 혼합해 새로운 작품을 창조하듯, 여러 가지 아이디어를 결합해 새로운 해결책을 만들어내는 능력을 표현합니다.
    
    총 5개의 비유 문장과 그에 대한 설명을 작성해주세요:
    """
    
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "당신은 입력된 역량과 자기소개서를 바탕으로 비유 문장을 생성하는 전문가입니다."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=700,  # 5개의 비유 문장과 설명을 위한 충분한 토큰 수 설정
        temperature=0.9,
    )

    # 결과 출력
    metaphor_sentences = response['choices'][0]['message']['content'].strip()
    return metaphor_sentences

- 자소서 1

In [7]:
# # 자소서 1
essay = "에너지 분야의 적자를 해소하고, 시민들에게 경제적이고 안전한 가스를 공급해드리고 싶어 KOGAS에 지원하게 되었습니다. 물류관리 수업에서 재고와 생산 프로세스 최적화 기법인 'Rock-Drill' 방식을 적용, 실제 산업 사례를 반영한 과제를 성공적으로 수행했습니다. 공급사슬 총괄 계획에서 인적자원, 초과시간, 재고비용, 생산 수를 조절하며 최적의 해를 찾는 연습을 수행했습니다. 요소 간의 균형을 이해하며 분석적 사고 능력을 발전시켰습니다. 나아가 마케팅 캠페인, 규모의 경제, 시간 관리와 같이 불확실한 요소를 고려해 최소 비용 문제를 해결하는 깊이 있는 과제를 완수했습니다. 이렇게 쌓은 제 능력을 가스 수요관리 분야에 적용시킬 수 있습니다. 지역별, 시기별 가스 수요를 조사하여 요소들을 고려하고, 최적의 해를 도출하는 방향으로 활용할 수 있습니다. 이러한 역량을 통해, KOGAS의 적자 해소 및 시민들의 효용 제고에 도움을 주고자 지원했습니다."
competency = "수리적/논리적 능력"

In [30]:
# temperature=0.7

sentence = gpt_competency_essay(competency, essay)
print("역량 및 자소서:\n", sentence)

역량 및 자소서:
 - 비유 문장: 저는 여러 가지 변수를 고려하며 최적의 결과를 도출하는 세심한 체스 플레이어입니다.
  설명: 체스 플레이어는 다양한 가능성을 고려하며 최고의 수를 두는 것처럼, 제가 수리적/논리적 능력을 활용하여 다양한 요소를 고려하고 최적의 결과를 도출하는 능력을 나타냅니다.

- 비유 문장: 저는 복잡한 문제를 분석하고 해결하는 능력을 가진 퍼즐의 마스터입니다.
  설명: 퍼즐을 완성하는 것은 복잡한 문제를 분석하고 해결하는 과정과 같습니다. 이는 제가 물류관리 수업에서 'Rock-Drill' 방식을 적용하여 실제 산업 사례를 반영한 과제를 성공적으로 수행한 것을 보여줍니다.

- 비유 문장: 저는 다양한 요소를 고려하여 최적의 가스 공급 계획을 세우는 전략가입니다.
  설명: 전략가는 여러 가지 상황을 고려하여 최적의 전략을 세우는 것처럼, 저는 수요관리 분야에서 다양한 요소를 고려하여 최적의 가스 공급 계획을 세우는 능력을 가지고 있습니다.

- 비유 문장: 저는 도전적인 과제를 해결하는 능력을 가진 수학의 마법사입니다.
  설명: 수학의 마법사는 복잡한 공식과 문제를 해결하는 능력을 가진 것처럼, 저는 마케팅 캠페인, 규모의 경제, 시간 관리 등 불확실한 요소를 고려하여 최소 비용 문제를 해결하는 깊이 있는 과제를 완수했습니다.

- 비유 문장: 저는 KOGAS의 시민들에게 가치를 제공하는 에너지 변환장치입니다.
  설명: 에너지 변환장치는 원자재를 유용한 에너지로 변환하는 것처럼, 저는 제 능력을 가스 수요관리에 적용하여 KOGAS의 적자 해소와 시민들의 효용 제고에


In [8]:
# temperature=0.9

sentence = gpt_competency_essay(competency, essay)
print("역량 및 자소서:\n", sentence)

역량 및 자소서:
 - 비유 문장: 저는 다양한 변수를 고려하여 최적의 해결책을 찾아내는 체스 선수입니다.
  설명: 체스 선수는 게임을 진행하면서 다양한 판세를 분석하고, 최선의 수를 두는 능력을 가지고 있습니다. 자기소개서를 작성한 사람도 마찬가지로 다양한 요소를 고려하여 문제 해결을 진행했다는 점에서 체스 선수로 비유했습니다.

- 비유 문장: 저는 정확한 계획과 조율을 통해 목표를 이루는 오케스트라의 지휘자입니다.
  설명: 오케스트라의 지휘자는 다양한 악기와 연주자들을 조율하여 하나의 음악 조화를 이루는 역할을 합니다. 이러한 특징은 자기소개서를 작성한 사람이 다양한 변수를 고려하여 최적의 가스 수요를 도출하려 한 점과 맞닿아 있습니다.

- 비유 문장: 저는 모든 요소를 고려해 최적화된 경로를 찾아내는 네비게이션입니다.
  설명: 네비게이션은 도로 상황, 거리, 시간 등을 고려해서 최적화된 경로를 제시합니다. 이는 자기소개서를 작성한 사람이 다양한 요소를 고려하며 최적의 해를 도출하려는 노력과 일치합니다.

- 비유 문장: 저는 불확실성 속에서도 명확한 판단을 내릴 수 있는 안개 속의 등대입니다.
  설명: 등대는 안개가 심한 바다에서도 항해하는 선박에게 길을 안내하는 역할을 합니다. 이것은 자기소개서를 작성한 사람이 불확실한 요소들 속에서도 정확한 판단을 내려 문제를 해결해 나간 모습과 같습니다.

- 비유 문장: 저는 복잡한 문제를 간단하게 풀어내는 루빅스 큐브의 마스터입니다.
  설명: 루빅스 큐브의 마스터는 복잡한 문제 상황에서도 간단하게 해결책을 찾아냅니다. 이것은


- 자소서 2

In [9]:
# 자소서 2
essay = "호기심이 많은 성향은 수업을 넘어 실생활에도 적용되었습니다. 그중 컴퓨터의 성능 향상을 위해 메모리 카드를 구매한 뒤, 제품의 성능이 국제 표준과 일치하는지 직접 평가했던 경험은 제품 시험 엔지니어로서의 성장 초석을 다져주었습니다. 인터넷에서 찾은 메모리 카드 테스트 방법에는 전공 수업 때 배운 용어와 개념이 수두룩했습니다. 학교 과제가 아니라, 스스로 전공지식을 제대로 적용하고 배워보고 자동기들을 모아 프로젝트로 추진했습니다. 먼저, 실제 메모리 카드 제조사의 제품 테스트 과정을 벤치마크하기 위해 관련 직무 현직자를 수소문했습니다. 며칠간 연락을 시도한 끝에 운좋게도 현직 제품 시험 엔지니어분과 연락이 닿아 실제 테스트 과정을 배울 수 있었습니다. 이후 상용 프로그램 사용법을 익히고 배운 과정을 실행해 보았습니다. 간이 챔버를 구성해 테스트를 진행하니, 온도 Stress를 주었을 때 특정 주소에서 반복적으로 에러가 발생하는 것을 확인했습니다. 저희 팀은 고온에서 발생할 수 있는 리스크를 조사하여 메모리 카드의 Capacitance가 고온에서 누설 전류가 많아지는 사실을 알았습니다. 이를 바탕으로 고온에서 발생한 에러가 ‘누설 전류에 의한 leakage성 불량일 것이다.’라는 가설을 세웠고, FW 설정에서 Refresh 주기를 절반으로 줄이며 가설을 검증할 수 있었습니다. 이후, 메모리 카드의 국제 표준화 기구의 데이터 시트를 확인하여 제조사가 품질 보증을 위해 준수해야 하는 표준이 존재함을 확인했고, 해당 기준상 특정 온도 이상에서의 불량은 제조사 품질 보증에는 문제가 없음을 확인했습니다. 단순 호기심으로 시작했던 제품 평가가 국제 표준을 확인하고, 데이터 시트를 통해 기준을 충족하는지 분석한 경험으로 이어졌습니다. 배터리 제조사 또한 국제 표준이 존재하고, 이와 같은 경험과 성향은 제품 시험 엔지니어에게 큰 장점이 될 것입니다."
competency = "호기심을 바탕으로 한 문제 해결 능력"

In [10]:
# temperature=0.9

sentence = gpt_competency_essay(competency, essay)
print("역량 및 자소서:\n", sentence)

역량 및 자소서:
 1. 비유 문장: 저는 끈질긴 호기심을 가지고 문제의 본질을 파헤치는 뚤린 쇳덩이입니다.
   설명: 쇳덩이는 물리적인 막을 뚫고 나아가는 힘을 의미하고, 이는 자기소개서에서 본인이 호기심을 바탕으로 문제를 파악하고 해결해나가는 과정을 상징합니다.

2. 비유 문장: 저는 신비로운 세계를 탐험하는 우주 탐사선입니다.
   설명: 우주 탐사선은 알려지지 않은 우주를 탐험하며 새로운 발견을 하는 것과 같이, 본인이 컴퓨터의 성능과 국제 표준에 대해 깊게 탐구하고 이해하는 과정을 나타냅니다.

3. 비유 문장: 저는 문제를 근본적으로 해결하는 능력을 가지고 있는 섬세한 외과 의사입니다.
   설명: 외과 의사가 복잡한 수술을 통해 문제를 해결하는 것처럼, 본인은 끊임없는 호기심과 문제해결 능력으로 메모리 카드의 문제를 근본적으로 파악하고 해결해냈습니다.

4. 비유 문장: 저는 호기심과 문제해결 능력을 가진것은 밝은 등대와 같습니다.
   설명: 등대는 어두운 밤도 밝게 비춰 길을 안내하는 역할을 하는 것처럼, 본인의 호기심과 문제해결 능력은 복잡한 문제를 명확하게 이해하고 해결하는 데 도움이 됩니다.

5. 비유 문장: 저는 끊임없는 호기심을 가지고 문제를 파헤치는 가혹한 산악인입니다.
   설명: 산악인은 어려운 산을 정복하기 위해 끊임없는 노력과 준비를 하는 것처럼, 본인은 복잡한 문제를 해결하기 위해 끊임없는 호기심과 노력을 가지고 있습니다.


# **KoGPT**

In [3]:
# KoGPT2 모델과 토크나이저 로드
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>')

c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be 

## KoGPT 기본 인퍼런스

In [4]:
## 기본 인퍼런스

# 기본 문장 생성을 위한 함수
def kogpt_inference(prompt, max_length=150):
    
    # 프롬프트를 토큰화하여 입력
    input_ids = tokenizer.encode(prompt, return_tensors="pt", max_length=500)

    # KoGPT2 모델을 사용하여 문장 생성
    output = model.generate(
        input_ids,
        max_length=max_length,  # 생성될 문장의 최대 길이
        num_return_sequences=1,  # 생성할 문장의 수
        no_repeat_ngram_size=2,  # 같은 단어가 반복되지 않도록 설정
        do_sample=True,          # 샘플링을 통해 다양성 확보
        top_k=50,                # 상위 50개 단어 중에서 선택
        top_p=0.95,              # 확률이 95%인 단어들 중에서 선택
        temperature=0.7,         # 창의성 조절 (높을수록 다양성 증가)
        max_new_tokens=80,      # 생성할 새로운 토큰 수 제한
        eos_token_id=tokenizer.eos_token_id  # 문장 종료를 위한 토큰 ID
    )

    # 생성된 문장 디코딩 및 출력
    generated_sentence = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_sentence

In [5]:
# 테스트용 프롬프트
prompt1 = "오늘은 날씨가 좋고, 나는"
generated_sentence1 = kogpt_inference(prompt1)
print(generated_sentence1)
print(" ")
prompt2 = "저는 여러 아이디어와 의견을 모아 창의적인 해결책을 만들어내는"
generated_sentence2 = kogpt_inference(prompt2)
print(generated_sentence2)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=80) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=80) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


오늘은 날씨가 좋고, 나는 오늘 밤 잠을 못 자겠습니다.
전국에서 미세먼지 주의보도 강화됐습니다.
서울은 미세 먼지 농도 보통 수준이지만 대구는 오전에 나쁨 단계까지 오를 것으로 보입니다.
오늘 낮부터는 기온이 오르면서 평년 기온을 회복하겠습니다.
오늘은 전국적으로 구름 많은 날씨를 보이겠고요,
동해안과 경남 동해안은 낮까지 빗방울이 떨어지는 곳이 있겠습니다.
현재 기온은 서울이
 
저는 여러 아이디어와 의견을 모아 창의적인 해결책을 만들어내는 것이 매우 중요하다고 생각해 왔다”고 말했다. 서울중앙지검 특수1부(부장검사 임관혁)는 19일 오전 9시30분께 서울 강남구 대치동 은마아파트 재건축조합 사무실 등을 압수수색했다.
검찰은 전날 압수한 자료를 분석해 은마를 재건축한 은마와 관련해 조합 관계자 등 30여 명을 체포해 조사 중이다.
검찰 관계자는 "압수수색을 통해 확보한 자료를 분석


## KoGPT 기본 프롬프트

### 1. 키워드만 입력

In [6]:
# 비유 문장과 설명을 함께 여러 개 생성하는 함수
def kogpt2_keywords(keywords):
    
    # KoGPT2에 전달할 프롬프트
    prompt = f"""
    주어진 키워드를 바탕으로, 자기소개서를 쓴 사람을 나타낼 수 있는 비유 문장 5개를 각각 '저는 ~입니다.' 형식으로 작성하고, 왜 그 비유를 사용했는지 구체적으로 설명해주세요.
    비유된 대상(예: 나침반)이 선택된 이유와 그 대상이 어떻게 주어진 키워드들과 연결되는지를 완성된 문장으로 설명해주세요.
    각 비유 문장은 반드시 '저는 ~입니다.'로 시작해야 하며, 간결하게 작성되어야 합니다. 총 5개의 비유 문장과 그에 대한 설명을 각각 작성해주세요.
    문장은 반드시 하나의 문장으로 완성되어야 합니다.

    예시:
    - 비유 문장: 저는 어떠한 문제도 분석하고 해결할 수 있는 확고한 방향성을 가진 나침반입니다.
      설명: 나침반은 방향을 제시하고, 문제 해결 능력은 올바른 방향을 찾아내는 것과 같기 때문에 나침반으로 비유했습니다. 나침반은 모든 상황에서 일관되게 목표를 향해 움직일 수 있는 도구로, 이 능력은 복잡한 문제 속에서도 명확한 방향을 찾는 능력과 일치합니다.

    - 비유 문장: 저는 여러 아이디어와 의견을 모아 창의적인 해결책을 만들어내는 팔레트입니다.
      설명: 팔레트는 다양한 색을 혼합해 새로운 작품을 창조하듯, 여러 가지 아이디어를 결합해 새로운 해결책을 만들어내는 능력을 표현합니다. 이 비유는 팔레트의 창의적인 역할이 다양한 의견을 효과적으로 융합하는 능력과 연관되기 때문에 사용되었습니다.

    5개의 비유 문장과 그에 대한 설명을 각각 작성해주세요. 각 문장들은 반드시 완전한 문장으로 끝나야 합니다.
    """
    
    # KoGPT2로 텍스트를 생성하기 위한 인풋
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    
    # KoGPT2 모델을 사용하여 텍스트 생성
    outputs = model.generate(
        inputs["input_ids"],
        max_length=700,  # 생성될 최대 토큰 수
        num_return_sequences=1,  # 생성할 시퀀스의 개수
        no_repeat_ngram_size=2,  # 반복을 방지
        repetition_penalty=1.5,  # 문장 반복에 대한 페널티
        temperature=0.9,  # 출력의 다양성 조절
        top_p=1.0,  # 확률에 기반한 선택
        pad_token_id=tokenizer.eos_token_id  # 패딩을 위한 토큰 ID 설정
    )
    
    # 결과 출력
    metaphor = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return metaphor

- 자소서 1

In [ ]:
# 자소서 1
# 자소서: 에너지 분야의 적자를 해소하고, 시민들에게 경제적이고 안전한 가스를 공급해드리고 싶어 KOGAS에 지원하게 되었습니다. 물류관리 수업에서 재고와 생산 프로세스 최적화 기법인 'Rock-Drill' 방식을 적용, 실제 산업 사례를 반영한 과제를 성공적으로 수행했습니다. 공급사슬 총괄 계획에서 인적자원, 초과시간, 재고비용, 생산 수를 조절하며 최적의 해를 찾는 연습을 수행했습니다. 요소 간의 균형을 이해하며 분석적 사고 능력을 발전시켰습니다. 나아가 마케팅 캠페인, 규모의 경제, 시간 관리와 같이 불확실한 요소를 고려해 최소 비용 문제를 해결하는 깊이 있는 과제를 완수했습니다. 이렇게 쌓은 제 능력을 가스 수요관리 분야에 적용시킬 수 있습니다. 지역별, 시기별 가스 수요를 조사하여 요소들을 고려하고, 최적의 해를 도출하는 방향으로 활용할 수 있습니다. 이러한 역량을 통해, KOGAS의 적자 해소 및 시민들의 효용 제고에 도움을 주고자 지원했습니다.
# 역량: 수리적/논리적 능력

In [7]:
# temperature=0.9, stop=["\n\n"]

keywords = ["분석적 사고", "최적화", "균형", "문제 해결", "효율성"]
sentence = kogpt2_keywords(keywords)
print(sentence)

c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



    주어진 키워드를 바탕으로, 자기소개서를 쓴 사람을 나타낼 수 있는 비유 문장 5개를 각각 '저는 ~입니다.' 형식으로 작성하고, 왜 그 비유를 사용했는지 구체적으로 설명해주세요.
    비유된 대상(예: 나침반)이 선택된 이유와 그 대상이 어떻게 주어진 키워드들과 연결되는지를 완성된 문장으로 설명해주세요.
    각 비유 문장은 반드시 '저는 ~입니다.'로 시작해야 하며, 간결하게 작성되어야 합니다. 총 5개의 비유 문장과 그에 대한 설명을 각각 작성해주세요.
    문장은 반드시 하나의 문장으로 완성되어야 합니다.

    예시:
    - 비유 문장: 저는 어떠한 문제도 분석하고 해결할 수 있는 확고한 방향성을 가진 나침반입니다.
      설명: 나침반은 방향을 제시하고, 문제 해결 능력은 올바른 방향을 찾아내는 것과 같기 때문에 나침반으로 비유했습니다. 나침반은 모든 상황에서 일관되게 목표를 향해 움직일 수 있는 도구로, 이 능력은 복잡한 문제 속에서도 명확한 방향을 찾는 능력과 일치합니다.

    - 비유 문장: 저는 여러 아이디어와 의견을 모아 창의적인 해결책을 만들어내는 팔레트입니다.
      설명: 팔레트는 다양한 색을 혼합해 새로운 작품을 창조하듯, 여러 가지 아이디어를 결합해 새로운 해결책을 만들어내는 능력을 표현합니다. 이 비유는 팔레트의 창의적인 역할이 다양한 의견을 효과적으로 융합하는 능력과 연관되기 때문에 사용되었습니다.

    5개의 비유 문장과 그에 대한 설명을 각각 작성해주세요. 각 문장들은 반드시 완전한 문장으로 끝나야 합니다.
    | _  : (예컨대) 어떤 문제가 발생했을 때 가장 먼저 떠오르는 아이디어에 대해 설명하는 것입니다.
이때 중요한 것은 바로 자신의 생각을 표현하는 능력인데, 이는 자신이 생각하는 바를 표현할 줄 아는 능력이죠.
또한 다른 사람의 의견이나 주장을 반박할 때도 마찬가지랍니다.
예를 들어 "내가 원하는 것이 무엇입니까?"라는 질문에 답하면 다음과 같은 대답을 할 수가 있습니다.
"나는

- 자소서 2

In [ ]:
# 자소서 2
# 자소서: 호기심이 많은 성향은 수업을 넘어 실생활에도 적용되었습니다. 그중 컴퓨터의 성능 향상을 위해 메모리 카드를 구매한 뒤, 제품의 성능이 국제 표준과 일치하는지 직접 평가했던 경험은 제품 시험 엔지니어로서의 성장 초석을 다져주었습니다. 인터넷에서 찾은 메모리 카드 테스트 방법에는 전공 수업 때 배운 용어와 개념이 수두룩했습니다. 학교 과제가 아니라, 스스로 전공지식을 제대로 적용하고 배워보고 자동기들을 모아 프로젝트로 추진했습니다. 먼저, 실제 메모리 카드 제조사의 제품 테스트 과정을 벤치마크하기 위해 관련 직무 현직자를 수소문했습니다. 며칠간 연락을 시도한 끝에 운좋게도 현직 제품 시험 엔지니어분과 연락이 닿아 실제 테스트 과정을 배울 수 있었습니다. 이후 상용 프로그램 사용법을 익히고 배운 과정을 실행해 보았습니다. 간이 챔버를 구성해 테스트를 진행하니, 온도 Stress를 주었을 때 특정 주소에서 반복적으로 에러가 발생하는 것을 확인했습니다. 저희 팀은 고온에서 발생할 수 있는 리스크를 조사하여 메모리 카드의 Capacitance가 고온에서 누설 전류가 많아지는 사실을 알았습니다. 이를 바탕으로 고온에서 발생한 에러가 ‘누설 전류에 의한 leakage성 불량일 것이다.’라는 가설을 세웠고, FW 설정에서 Refresh 주기를 절반으로 줄이며 가설을 검증할 수 있었습니다. 이후, 메모리 카드의 국제 표준화 기구의 데이터 시트를 확인하여 제조사가 품질 보증을 위해 준수해야 하는 표준이 존재함을 확인했고, 해당 기준상 특정 온도 이상에서의 불량은 제조사 품질 보증에는 문제가 없음을 확인했습니다. 단순 호기심으로 시작했던 제품 평가가 국제 표준을 확인하고, 데이터 시트를 통해 기준을 충족하는지 분석한 경험으로 이어졌습니다. 배터리 제조사 또한 국제 표준이 존재하고, 이와 같은 경험과 성향은 제품 시험 엔지니어에게 큰 장점이 될 것입니다.
# 역량: 호기심을 바탕으로 한 문제 해결 능력

In [9]:
# temperature=0.9

# 직접 키워드:
keywords = ["호기심", "전공지식", "벤치마킹", "가설 검증", "국제 표준"]
sentence = kogpt2_keywords(keywords)
print("직접 키워드 사용:\n", sentence)
print("")
print("")
# 간접 키워드:
keywords = ["자기주도성", "전공지식 응용", "분석력", "탐구심", "실험적 사고"]
sentence = kogpt2_keywords(keywords)
print("간접 키워드 사용:\n", sentence)

직접 키워드 사용:
 
    주어진 키워드를 바탕으로, 자기소개서를 쓴 사람을 나타낼 수 있는 비유 문장 5개를 각각 '저는 ~입니다.' 형식으로 작성하고, 왜 그 비유를 사용했는지 구체적으로 설명해주세요.
    비유된 대상(예: 나침반)이 선택된 이유와 그 대상이 어떻게 주어진 키워드들과 연결되는지를 완성된 문장으로 설명해주세요.
    각 비유 문장은 반드시 '저는 ~입니다.'로 시작해야 하며, 간결하게 작성되어야 합니다. 총 5개의 비유 문장과 그에 대한 설명을 각각 작성해주세요.
    문장은 반드시 하나의 문장으로 완성되어야 합니다.

    예시:
    - 비유 문장: 저는 어떠한 문제도 분석하고 해결할 수 있는 확고한 방향성을 가진 나침반입니다.
      설명: 나침반은 방향을 제시하고, 문제 해결 능력은 올바른 방향을 찾아내는 것과 같기 때문에 나침반으로 비유했습니다. 나침반은 모든 상황에서 일관되게 목표를 향해 움직일 수 있는 도구로, 이 능력은 복잡한 문제 속에서도 명확한 방향을 찾는 능력과 일치합니다.

    - 비유 문장: 저는 여러 아이디어와 의견을 모아 창의적인 해결책을 만들어내는 팔레트입니다.
      설명: 팔레트는 다양한 색을 혼합해 새로운 작품을 창조하듯, 여러 가지 아이디어를 결합해 새로운 해결책을 만들어내는 능력을 표현합니다. 이 비유는 팔레트의 창의적인 역할이 다양한 의견을 효과적으로 융합하는 능력과 연관되기 때문에 사용되었습니다.

    5개의 비유 문장과 그에 대한 설명을 각각 작성해주세요. 각 문장들은 반드시 완전한 문장으로 끝나야 합니다.
    | _  : (예컨대) 어떤 문제가 발생했을 때 가장 먼저 떠오르는 아이디어에 대해 설명하는 것입니다.
이때 중요한 것은 바로 자신의 생각을 표현하는 능력인데, 이는 자신이 생각하는 바를 표현할 줄 아는 능력이죠.
또한 다른 사람의 의견이나 주장을 반박할 때도 마찬가지랍니다.
예를 들어 "내가 원하는 것이 무엇입니까?"라는 질문에 답하면 다음과 같은 대답을 할 

### 2. 키워드 & 자기소개서 입력

In [10]:
def kogpt2_keywords_essay(keywords, essay):
    
    # KoGPT2에 전달할 프롬프트
    prompt = f"""
    주어진 키워드: {keywords}
    자기소개서 내용: {essay}
    위의 키워드와 자기소개서를 바탕으로, 자기소개서를 쓴 사람을 나타낼 수 있는 비유 문장 5개를 각각 '저는 ~입니다.' 형식으로 작성하고, 왜 그 비유를 사용했는지 구체적으로 설명해주세요.
    비유된 대상(예: 나침반)이 선택된 이유와 그 대상이 어떻게 주어진 키워드들과 자기소개서 내용과 연결되는지를 완성된 문장으로 설명해주세요.
    각 비유 문장은 반드시 '저는 ~입니다.'로 시작해야 하며, 간결하게 작성되어야 합니다. 총 5개의 비유 문장과 그에 대한 설명을 각각 작성해주세요.
    문장은 반드시 하나의 문장으로 완성되어야 합니다.

    예시:
    - 비유 문장: 저는 어떠한 문제도 분석하고 해결할 수 있는 확고한 방향성을 가진 나침반입니다.
      설명: 나침반은 방향을 제시하고, 문제 해결 능력은 올바른 방향을 찾아내는 것과 같기 때문에 나침반으로 비유했습니다. 나침반은 모든 상황에서 일관되게 목표를 향해 움직일 수 있는 도구로, 이 능력은 복잡한 문제 속에서도 명확한 방향을 찾는 능력과 일치합니다.

    - 비유 문장: 저는 여러 아이디어와 의견을 모아 창의적인 해결책을 만들어내는 팔레트입니다.
      설명: 팔레트는 다양한 색을 혼합해 새로운 작품을 창조하듯, 여러 가지 아이디어를 결합해 새로운 해결책을 만들어내는 능력을 표현합니다. 이 비유는 팔레트의 창의적인 역할이 다양한 의견을 효과적으로 융합하는 능력과 연관되기 때문에 사용되었습니다.

    5개의 비유 문장과 그에 대한 설명을 각각 작성해주세요. 각 문장들은 반드시 완전한 문장으로 끝나야 해.
    """
    
    # KoGPT2로 텍스트를 생성하기 위한 인풋
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    
    # KoGPT2 모델을 사용하여 텍스트 생성
    outputs = model.generate(
        inputs["input_ids"],
        max_length=700,  # 생성될 최대 토큰 수
        num_return_sequences=1,  # 생성할 시퀀스의 개수
        no_repeat_ngram_size=2,  # 반복을 방지
        repetition_penalty=1.5,  # 문장 반복에 대한 페널티
        temperature=0.9,  # 출력의 다양성 조절
        top_p=1.0,  # 확률에 기반한 선택
        pad_token_id=tokenizer.eos_token_id  # 패딩을 위한 토큰 ID 설정
    )
    
    # 결과 출력
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text


- 자소서 1

In [11]:
# 자소서 1
# 자소서: 에너지 분야의 적자를 해소하고, 시민들에게 경제적이고 안전한 가스를 공급해드리고 싶어 KOGAS에 지원하게 되었습니다. 물류관리 수업에서 재고와 생산 프로세스 최적화 기법인 'Rock-Drill' 방식을 적용, 실제 산업 사례를 반영한 과제를 성공적으로 수행했습니다. 공급사슬 총괄 계획에서 인적자원, 초과시간, 재고비용, 생산 수를 조절하며 최적의 해를 찾는 연습을 수행했습니다. 요소 간의 균형을 이해하며 분석적 사고 능력을 발전시켰습니다. 나아가 마케팅 캠페인, 규모의 경제, 시간 관리와 같이 불확실한 요소를 고려해 최소 비용 문제를 해결하는 깊이 있는 과제를 완수했습니다. 이렇게 쌓은 제 능력을 가스 수요관리 분야에 적용시킬 수 있습니다. 지역별, 시기별 가스 수요를 조사하여 요소들을 고려하고, 최적의 해를 도출하는 방향으로 활용할 수 있습니다. 이러한 역량을 통해, KOGAS의 적자 해소 및 시민들의 효용 제고에 도움을 주고자 지원했습니다.
# 역량: 수리적/논리적 능력

In [17]:
essay = "에너지 분야의 적자를 해소하고, 시민들에게 경제적이고 안전한 가스를 공급해드리고 싶어 KOGAS에 지원하게 되었습니다. 물류관리 수업에서 재고와 생산 프로세스 최적화 기법인 'Rock-Drill' 방식을 적용, 실제 산업 사례를 반영한 과제를 성공적으로 수행했습니다. 공급사슬 총괄 계획에서 인적자원, 초과시간, 재고비용, 생산 수를 조절하며 최적의 해를 찾는 연습을 수행했습니다. 요소 간의 균형을 이해하며 분석적 사고 능력을 발전시켰습니다. 나아가 마케팅 캠페인, 규모의 경제, 시간 관리와 같이 불확실한 요소를 고려해 최소 비용 문제를 해결하는 깊이 있는 과제를 완수했습니다. 이렇게 쌓은 제 능력을 가스 수요관리 분야에 적용시킬 수 있습니다. 지역별, 시기별 가스 수요를 조사하여 요소들을 고려하고, 최적의 해를 도출하는 방향으로 활용할 수 있습니다. 이러한 역량을 통해, KOGAS의 적자 해소 및 시민들의 효용 제고에 도움을 주고자 지원했습니다."
keywords = ["분석적 사고", "최적화", "균형", "문제 해결", "효율성"]

sentence = kogpt2_keywords_essay(keywords, essay)
print(sentence)


    주어진 키워드: ['분석적 사고', '최적화', '균형', '문제 해결', '효율성']
    자기소개서 내용: 에너지 분야의 적자를 해소하고, 시민들에게 경제적이고 안전한 가스를 공급해드리고 싶어 KOGAS에 지원하게 되었습니다. 물류관리 수업에서 재고와 생산 프로세스 최적화 기법인 'Rock-Drill' 방식을 적용, 실제 산업 사례를 반영한 과제를 성공적으로 수행했습니다. 공급사슬 총괄 계획에서 인적자원, 초과시간, 재고비용, 생산 수를 조절하며 최적의 해를 찾는 연습을 수행했습니다. 요소 간의 균형을 이해하며 분석적 사고 능력을 발전시켰습니다. 나아가 마케팅 캠페인, 규모의 경제, 시간 관리와 같이 불확실한 요소를 고려해 최소 비용 문제를 해결하는 깊이 있는 과제를 완수했습니다. 이렇게 쌓은 제 능력을 가스 수요관리 분야에 적용시킬 수 있습니다. 지역별, 시기별 가스 수요를 조사하여 요소들을 고려하고, 최적의 해를 도출하는 방향으로 활용할 수 있습니다. 이러한 역량을 통해, KOGAS의 적자 해소 및 시민들의 효용 제고에 도움을 주고자 지원했습니다.
    위의 키워드와 자기소개서를 바탕으로, 자기소개서를 쓴 사람을 나타낼 수 있는 비유 문장 5개를 각각 '저는 ~입니다.' 형식으로 작성하고, 왜 그 비유를 사용했는지 구체적으로 설명해주세요.
    비유된 대상(예: 나침반)이 선택된 이유와 그 대상이 어떻게 주어진 키워드들과 자기소개서 내용과 연결되는지를 완성된 문장으로 설명해주세요.
    각 비유 문장은 반드시 '저는 ~입니다.'로 시작해야 하며, 간결하게 작성되어야 합니다. 총 5개의 비유 문장과 그에 대한 설명을 각각 작성해주세요.
    문장은 반드시 하나의 문장으로 완성되어야 합니다.

    예시:
    - 비유 문장: 저는 어떠한 문제도 분석하고 해결할 수 있는 확고한 방향성을 가진 나침반입니다.
      설명: 나침반은 방향을 제시하고, 문제 해결 능력은 올바른 방향을 찾아내는 것과 같기 때문에 나침반으로 비유했습니다. 나

- 자소서 2

In [18]:
# 자소서 2
# 자소서: 호기심이 많은 성향은 수업을 넘어 실생활에도 적용되었습니다. 그중 컴퓨터의 성능 향상을 위해 메모리 카드를 구매한 뒤, 제품의 성능이 국제 표준과 일치하는지 직접 평가했던 경험은 제품 시험 엔지니어로서의 성장 초석을 다져주었습니다. 인터넷에서 찾은 메모리 카드 테스트 방법에는 전공 수업 때 배운 용어와 개념이 수두룩했습니다. 학교 과제가 아니라, 스스로 전공지식을 제대로 적용하고 배워보고 자동기들을 모아 프로젝트로 추진했습니다. 먼저, 실제 메모리 카드 제조사의 제품 테스트 과정을 벤치마크하기 위해 관련 직무 현직자를 수소문했습니다. 며칠간 연락을 시도한 끝에 운좋게도 현직 제품 시험 엔지니어분과 연락이 닿아 실제 테스트 과정을 배울 수 있었습니다. 이후 상용 프로그램 사용법을 익히고 배운 과정을 실행해 보았습니다. 간이 챔버를 구성해 테스트를 진행하니, 온도 Stress를 주었을 때 특정 주소에서 반복적으로 에러가 발생하는 것을 확인했습니다. 저희 팀은 고온에서 발생할 수 있는 리스크를 조사하여 메모리 카드의 Capacitance가 고온에서 누설 전류가 많아지는 사실을 알았습니다. 이를 바탕으로 고온에서 발생한 에러가 ‘누설 전류에 의한 leakage성 불량일 것이다.’라는 가설을 세웠고, FW 설정에서 Refresh 주기를 절반으로 줄이며 가설을 검증할 수 있었습니다. 이후, 메모리 카드의 국제 표준화 기구의 데이터 시트를 확인하여 제조사가 품질 보증을 위해 준수해야 하는 표준이 존재함을 확인했고, 해당 기준상 특정 온도 이상에서의 불량은 제조사 품질 보증에는 문제가 없음을 확인했습니다. 단순 호기심으로 시작했던 제품 평가가 국제 표준을 확인하고, 데이터 시트를 통해 기준을 충족하는지 분석한 경험으로 이어졌습니다. 배터리 제조사 또한 국제 표준이 존재하고, 이와 같은 경험과 성향은 제품 시험 엔지니어에게 큰 장점이 될 것입니다.
# 역량: 호기심을 바탕으로 한 문제 해결 능력

In [19]:
essay = "호기심이 많은 성향은 수업을 넘어 실생활에도 적용되었습니다. 그중 컴퓨터의 성능 향상을 위해 메모리 카드를 구매한 뒤, 제품의 성능이 국제 표준과 일치하는지 직접 평가했던 경험은 제품 시험 엔지니어로서의 성장 초석을 다져주었습니다. 인터넷에서 찾은 메모리 카드 테스트 방법에는 전공 수업 때 배운 용어와 개념이 수두룩했습니다. 학교 과제가 아니라, 스스로 전공지식을 제대로 적용하고 배워보고 자동기들을 모아 프로젝트로 추진했습니다. 먼저, 실제 메모리 카드 제조사의 제품 테스트 과정을 벤치마크하기 위해 관련 직무 현직자를 수소문했습니다. 며칠간 연락을 시도한 끝에 운좋게도 현직 제품 시험 엔지니어분과 연락이 닿아 실제 테스트 과정을 배울 수 있었습니다. 이후 상용 프로그램 사용법을 익히고 배운 과정을 실행해 보았습니다. 간이 챔버를 구성해 테스트를 진행하니, 온도 Stress를 주었을 때 특정 주소에서 반복적으로 에러가 발생하는 것을 확인했습니다. 저희 팀은 고온에서 발생할 수 있는 리스크를 조사하여 메모리 카드의 Capacitance가 고온에서 누설 전류가 많아지는 사실을 알았습니다. 이를 바탕으로 고온에서 발생한 에러가 ‘누설 전류에 의한 leakage성 불량일 것이다.’라는 가설을 세웠고, FW 설정에서 Refresh 주기를 절반으로 줄이며 가설을 검증할 수 있었습니다. 이후, 메모리 카드의 국제 표준화 기구의 데이터 시트를 확인하여 제조사가 품질 보증을 위해 준수해야 하는 표준이 존재함을 확인했고, 해당 기준상 특정 온도 이상에서의 불량은 제조사 품질 보증에는 문제가 없음을 확인했습니다. 단순 호기심으로 시작했던 제품 평가가 국제 표준을 확인하고, 데이터 시트를 통해 기준을 충족하는지 분석한 경험으로 이어졌습니다. 배터리 제조사 또한 국제 표준이 존재하고, 이와 같은 경험과 성향은 제품 시험 엔지니어에게 큰 장점이 될 것입니다."

# 직접 키워드:
keywords = ["호기심", "전공지식", "벤치마킹", "가설 검증", "국제 표준"]
sentence = kogpt2_keywords_essay(keywords, essay)
print("직접 키워드 사용:\n", sentence)
print("")
print("")
# 간접 키워드:
keywords = ["자기주도성", "전공지식 응용", "분석력", "탐구심", "실험적 사고"]
sentence = kogpt2_keywords_essay(keywords, essay)
print("간접 키워드 사용:\n", sentence)

직접 키워드 사용:
 
    주어진 키워드: ['호기심', '전공지식', '벤치마킹', '가설 검증', '국제 표준']
    자기소개서 내용: 호기심이 많은 성향은 수업을 넘어 실생활에도 적용되었습니다. 그중 컴퓨터의 성능 향상을 위해 메모리 카드를 구매한 뒤, 제품의 성능이 국제 표준과 일치하는지 직접 평가했던 경험은 제품 시험 엔지니어로서의 성장 초석을 다져주었습니다. 인터넷에서 찾은 메모리 카드 테스트 방법에는 전공 수업 때 배운 용어와 개념이 수두룩했습니다. 학교 과제가 아니라, 스스로 전공지식을 제대로 적용하고 배워보고 자동기들을 모아 프로젝트로 추진했습니다. 먼저, 실제 메모리 카드 제조사의 제품 테스트 과정을 벤치마크하기 위해 관련 직무 현직자를 수소문했습니다. 며칠간 연락을 시도한 끝에 운좋게도 현직 제품 시험 엔지니어분과 연락이 닿아 실제 테스트 과정을 배울 수 있었습니다. 이후 상용 프로그램 사용법을 익히고 배운 과정을 실행해 보았습니다. 간이 챔버를 구성해 테스트를 진행하니, 온도 Stress를 주었을 때 특정 주소에서 반복적으로 에러가 발생하는 것을 확인했습니다. 저희 팀은 고온에서 발생할 수 있는 리스크를 조사하여 메모리 카드의 Capacitance가 고온에서 누설 전류가 많아지는 사실을 알았습니다. 이를 바탕으로 고온에서 발생한 에러가 ‘누설 전류에 의한 leakage성 불량일 것이다.’라는 가설을 세웠고, FW 설정에서 Refresh 주기를 절반으로 줄이며 가설을 검증할 수 있었습니다. 이후, 메모리 카드의 국제 표준화 기구의 데이터 시트를 확인하여 제조사가 품질 보증을 위해 준수해야 하는 표준이 존재함을 확인했고, 해당 기준상 특정 온도 이상에서의 불량은 제조사 품질 보증에는 문제가 없음을 확인했습니다. 단순 호기심으로 시작했던 제품 평가가 국제 표준을 확인하고, 데이터 시트를 통해 기준을 충족하는지 분석한 경험으로 이어졌습니다. 배터리 제조사 또한 국제 표준이 존재하고, 이와 같은 경험과 성향은 제품 시험 엔지니어에게 큰 장점

### 3. 역량 & 자기소개서 입력

In [26]:
def kogpt2_competency_essay(competency, essay):
    
    # KoGPT2에 전달할 프롬프트
    prompt = f"""
    [역량]: {competency}
    [자기소개서 내용]: {essay}
    
    위의 역량과 자기소개서 내용을 바탕으로, 자기소개서를 쓴 사람을 나타낼 수 있는 비유 문장 5개를 작성하고, 각 비유 문장에 대해 왜 그 비유를 선택했는지 설명해주세요.
    각 문장은 '저는 ~입니다.' 형식으로 작성되어야 하며, 비유된 대상이 역량과 어떻게 연결되는지 구체적으로 설명해주세요.
    각 문장은 하나의 문장으로 완성되어야 하며, 설명은 문장 뒤에 추가되어야 합니다.

    예시:
    - 비유 문장: 저는 어떠한 문제도 분석하고 해결할 수 있는 확고한 방향성을 가진 나침반입니다.
      설명: 나침반은 방향을 제시하고, 문제 해결 능력은 올바른 방향을 찾아내는 것과 같기 때문에 나침반으로 비유했습니다.
    
    - 비유 문장: 저는 여러 아이디어와 의견을 모아 창의적인 해결책을 만들어내는 팔레트입니다.
      설명: 팔레트는 다양한 색을 혼합해 새로운 작품을 창조하듯, 여러 가지 아이디어를 결합해 새로운 해결책을 만들어내는 능력을 표현합니다.
    
    총 5개의 비유 문장과 그에 대한 설명을 작성해주세요:
    """
    
    # KoGPT2로 텍스트를 생성하기 위한 인풋
    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    
    # KoGPT2 모델을 사용하여 텍스트 생성
    outputs = model.generate(
        inputs["input_ids"],
        max_length=700,  # 생성될 최대 토큰 수
        num_return_sequences=1,  # 생성할 시퀀스의 개수
        no_repeat_ngram_size=2,  # 반복을 방지
        repetition_penalty=1.5,  # 문장 반복에 대한 페널티
        temperature=0.7,  # 출력의 다양성 조절
        top_p=1.0,  # 확률에 기반한 선택
        pad_token_id=tokenizer.eos_token_id  # 패딩을 위한 토큰 ID 설정
    )
    
    # 결과 출력
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

- 자소서 1

In [27]:
# 자소서 1
essay = "에너지 분야의 적자를 해소하고, 시민들에게 경제적이고 안전한 가스를 공급해드리고 싶어 KOGAS에 지원하게 되었습니다. 물류관리 수업에서 재고와 생산 프로세스 최적화 기법인 'Rock-Drill' 방식을 적용, 실제 산업 사례를 반영한 과제를 성공적으로 수행했습니다. 공급사슬 총괄 계획에서 인적자원, 초과시간, 재고비용, 생산 수를 조절하며 최적의 해를 찾는 연습을 수행했습니다. 요소 간의 균형을 이해하며 분석적 사고 능력을 발전시켰습니다. 나아가 마케팅 캠페인, 규모의 경제, 시간 관리와 같이 불확실한 요소를 고려해 최소 비용 문제를 해결하는 깊이 있는 과제를 완수했습니다. 이렇게 쌓은 제 능력을 가스 수요관리 분야에 적용시킬 수 있습니다. 지역별, 시기별 가스 수요를 조사하여 요소들을 고려하고, 최적의 해를 도출하는 방향으로 활용할 수 있습니다. 이러한 역량을 통해, KOGAS의 적자 해소 및 시민들의 효용 제고에 도움을 주고자 지원했습니다."
competency = "수리적/논리적 능력"

sentence = kogpt2_competency_essay(competency, essay)
print("역량 및 자소서:\n", sentence)

c:\Users\USER\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


역량 및 자소서:
 
    [역량]: 수리적/논리적 능력
    [자기소개서 내용]: 에너지 분야의 적자를 해소하고, 시민들에게 경제적이고 안전한 가스를 공급해드리고 싶어 KOGAS에 지원하게 되었습니다. 물류관리 수업에서 재고와 생산 프로세스 최적화 기법인 'Rock-Drill' 방식을 적용, 실제 산업 사례를 반영한 과제를 성공적으로 수행했습니다. 공급사슬 총괄 계획에서 인적자원, 초과시간, 재고비용, 생산 수를 조절하며 최적의 해를 찾는 연습을 수행했습니다. 요소 간의 균형을 이해하며 분석적 사고 능력을 발전시켰습니다. 나아가 마케팅 캠페인, 규모의 경제, 시간 관리와 같이 불확실한 요소를 고려해 최소 비용 문제를 해결하는 깊이 있는 과제를 완수했습니다. 이렇게 쌓은 제 능력을 가스 수요관리 분야에 적용시킬 수 있습니다. 지역별, 시기별 가스 수요를 조사하여 요소들을 고려하고, 최적의 해를 도출하는 방향으로 활용할 수 있습니다. 이러한 역량을 통해, KOGAS의 적자 해소 및 시민들의 효용 제고에 도움을 주고자 지원했습니다.
    
    위의 역량과 자기소개서 내용을 바탕으로, 자기소개서를 쓴 사람을 나타낼 수 있는 비유 문장 5개를 작성하고, 각 비유 문장에 대해 왜 그 비유를 선택했는지 설명해주세요.
    각 문장은 '저는 ~입니다.' 형식으로 작성되어야 하며, 비유된 대상이 역량과 어떻게 연결되는지 구체적으로 설명해주세요.
    각 문장은 하나의 문장으로 완성되어야 하며, 설명은 문장 뒤에 추가되어야 합니다.

    예시:
    - 비유 문장: 저는 어떠한 문제도 분석하고 해결할 수 있는 확고한 방향성을 가진 나침반입니다.
      설명: 나침반은 방향을 제시하고, 문제 해결 능력은 올바른 방향을 찾아내는 것과 같기 때문에 나침반으로 비유했습니다.
    
    - 비유 문장: 저는 여러 아이디어와 의견을 모아 창의적인 해결책을 만들어내는 팔레트입니다.
      설명: 팔레트는 다양한 색을 혼합해 새로운 작품을 창조하듯, 여러 가지

- 자소서 2

In [28]:
# 자소서 2
essay = "호기심이 많은 성향은 수업을 넘어 실생활에도 적용되었습니다. 그중 컴퓨터의 성능 향상을 위해 메모리 카드를 구매한 뒤, 제품의 성능이 국제 표준과 일치하는지 직접 평가했던 경험은 제품 시험 엔지니어로서의 성장 초석을 다져주었습니다. 인터넷에서 찾은 메모리 카드 테스트 방법에는 전공 수업 때 배운 용어와 개념이 수두룩했습니다. 학교 과제가 아니라, 스스로 전공지식을 제대로 적용하고 배워보고 자동기들을 모아 프로젝트로 추진했습니다. 먼저, 실제 메모리 카드 제조사의 제품 테스트 과정을 벤치마크하기 위해 관련 직무 현직자를 수소문했습니다. 며칠간 연락을 시도한 끝에 운좋게도 현직 제품 시험 엔지니어분과 연락이 닿아 실제 테스트 과정을 배울 수 있었습니다. 이후 상용 프로그램 사용법을 익히고 배운 과정을 실행해 보았습니다. 간이 챔버를 구성해 테스트를 진행하니, 온도 Stress를 주었을 때 특정 주소에서 반복적으로 에러가 발생하는 것을 확인했습니다. 저희 팀은 고온에서 발생할 수 있는 리스크를 조사하여 메모리 카드의 Capacitance가 고온에서 누설 전류가 많아지는 사실을 알았습니다. 이를 바탕으로 고온에서 발생한 에러가 ‘누설 전류에 의한 leakage성 불량일 것이다.’라는 가설을 세웠고, FW 설정에서 Refresh 주기를 절반으로 줄이며 가설을 검증할 수 있었습니다. 이후, 메모리 카드의 국제 표준화 기구의 데이터 시트를 확인하여 제조사가 품질 보증을 위해 준수해야 하는 표준이 존재함을 확인했고, 해당 기준상 특정 온도 이상에서의 불량은 제조사 품질 보증에는 문제가 없음을 확인했습니다. 단순 호기심으로 시작했던 제품 평가가 국제 표준을 확인하고, 데이터 시트를 통해 기준을 충족하는지 분석한 경험으로 이어졌습니다. 배터리 제조사 또한 국제 표준이 존재하고, 이와 같은 경험과 성향은 제품 시험 엔지니어에게 큰 장점이 될 것입니다."
competency = "호기심을 바탕으로 한 문제 해결 능력"

sentence = kogpt2_competency_essay(competency, essay)
print("역량 및 자소서:\n", sentence)

직접 키워드 사용:
 
    [역량]: ['호기심', '전공지식', '벤치마킹', '가설 검증', '국제 표준']
    [자기소개서 내용]: 호기심이 많은 성향은 수업을 넘어 실생활에도 적용되었습니다. 그중 컴퓨터의 성능 향상을 위해 메모리 카드를 구매한 뒤, 제품의 성능이 국제 표준과 일치하는지 직접 평가했던 경험은 제품 시험 엔지니어로서의 성장 초석을 다져주었습니다. 인터넷에서 찾은 메모리 카드 테스트 방법에는 전공 수업 때 배운 용어와 개념이 수두룩했습니다. 학교 과제가 아니라, 스스로 전공지식을 제대로 적용하고 배워보고 자동기들을 모아 프로젝트로 추진했습니다. 먼저, 실제 메모리 카드 제조사의 제품 테스트 과정을 벤치마크하기 위해 관련 직무 현직자를 수소문했습니다. 며칠간 연락을 시도한 끝에 운좋게도 현직 제품 시험 엔지니어분과 연락이 닿아 실제 테스트 과정을 배울 수 있었습니다. 이후 상용 프로그램 사용법을 익히고 배운 과정을 실행해 보았습니다. 간이 챔버를 구성해 테스트를 진행하니, 온도 Stress를 주었을 때 특정 주소에서 반복적으로 에러가 발생하는 것을 확인했습니다. 저희 팀은 고온에서 발생할 수 있는 리스크를 조사하여 메모리 카드의 Capacitance가 고온에서 누설 전류가 많아지는 사실을 알았습니다. 이를 바탕으로 고온에서 발생한 에러가 ‘누설 전류에 의한 leakage성 불량일 것이다.’라는 가설을 세웠고, FW 설정에서 Refresh 주기를 절반으로 줄이며 가설을 검증할 수 있었습니다. 이후, 메모리 카드의 국제 표준화 기구의 데이터 시트를 확인하여 제조사가 품질 보증을 위해 준수해야 하는 표준이 존재함을 확인했고, 해당 기준상 특정 온도 이상에서의 불량은 제조사 품질 보증에는 문제가 없음을 확인했습니다. 단순 호기심으로 시작했던 제품 평가가 국제 표준을 확인하고, 데이터 시트를 통해 기준을 충족하는지 분석한 경험으로 이어졌습니다. 배터리 제조사 또한 국제 표준이 존재하고, 이와 같은 경험과 성향은 제품 시험 엔지니어에게 큰 장점이